<a href="https://colab.research.google.com/github/JordanByrd2380/text-to-image-GAN/blob/main/cGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

 cGAN.ipynb		 idxtoembed.pkl
 cs231n			 idxtotext.pkl
'Data Collection.gdoc'	'Introduction and Literature Review.gdoc'
 embeddings_index.pkl	 isbntocsv.pkl
'Final Book Bundles'	 isbntoimg.pkl
'Getting started.pdf'	'Jordan RP Rehearsal Feedback Form.gslides'
 glove.6B.100d.txt	 __pycache__
 glove.6B.50d.txt	'Results Conclusion.gdoc'
 helper.py


In [ ]:
import numpy as np
import torch
import os
import pandas as pd
from torchvision import datasets, transforms
import matplotlib.pyplot as plt



rootdir = "/content/drive/My Drive"
rootdir = os.path.join(rootdir, "Final Book Bundles")
transform = transforms.Compose([#transforms.Resize(65),
                                            #transforms.CenterCrop(64),
                                            transforms.Resize(257),
                                            transforms.CenterCrop(256),
                                           transforms.ToTensor()])
dataset = datasets.ImageFolder(rootdir, transform=transform)
#dataset = datasets.ImageFolder(rootdir)
ds = {}
csvs = {}

print(len(dataset))

4639


In [ ]:
import torch
index = 0
for pic in dataset:
  if torch.count_nonzero(pic[0]- 1) > 49152:
    ds[index] = pic[0]
    index += 1
print(len(ds))
#counter = 0
#ds2 = {}
#counter = 0
#for key in ds:
#  for elem in ds[key]:
#    if torch.count_nonzero(elem[0]- 1) > 3072:
#      ds2[counter] = elem[0]
#      counter += 1

4060


In [ ]:
import pickle
with open('ds.pkl', 'wb') as fid:
     pickle.dump(ds, fid)

In [ ]:
import os
helper_loc = "/content/drive/My Drive"
import sys
sys.path.append(os.path.abspath(helper_loc))
import helper


In [ ]:
#counter = 0
#for i in range(4639):
#  if torch.count_nonzero(dataset[i][0] - 1) != 0:
#    counter += 1
#  if counter % 100 == 0:
#    print(counter)
#print(counter)

print(dataset.type)

#dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=50)
#torch.save(dataset, 'dataset2.pt')

#images, labels = next(iter(dataloader))
#helper.imshow(images[-1], normalize=False)
#print(images[-1].shape)

AttributeError: ignored

In [ ]:
idxtoimage= torch.zeros((4639, 3, 64, 64))
for i in range(4639):
  if i % 100 == 0:
    print(i)
  idxtoimage[i] = dataset[i][0]

torch.save(idxtoimage, 'tensor.pt')
#idxtoimage2[:] = dataset[:][0]
#print(dataset[:, 0].shape)
#print(len(dataset))
#print(dataset[0])
#images, idx = dataset
#print(images.shape)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600


In [ ]:
from torch.utils.data import Dataset, DataLoader
class ImageDataset(Dataset):
  def __init__(self, tensor):
    self.tensor = tensor
  def __len__(self):
    return self.tensor.shape[0]
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    return self.tensor[idx]

tens = torch.ones((100, 3, 64, 64))
dataset = ImageDataset(tens)
dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=50)
for sample in dataloader:
    right_images = sample
    print(right_images.shape)

torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])


In [ ]:
import numpy as np
import torch
import os
import pandas as pd
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

rootdir = "/content/drive/My Drive"
rootdir = os.path.join(rootdir, "Final Book Bundles")


csvs = {}
for subdir, dirs, files in os.walk(rootdir):
    if subdir.startswith("/content/drive/My Drive/Final Book Bundles/") and not subdir.endswith("Im") and not subdir.endswith("a"):
      for file in files:
        if file.endswith(".csv") and not file.endswith("MD.csv") and not file.startswith("MD"):
          booklines = pd.read_csv(os.path.join(subdir, file))
          csvs[file[:-4]] = booklines


In [ ]:
 import pickle
#with open('isbntocsv.pkl', 'wb') as fid:
#     pickle.dump(csvs, fid)
with open('drive/MyDrive/isbntoimg.pkl', 'rb') as fid:
     ds = pickle.load(fid)

In [ ]:
from PIL import Image
import torchvision.transforms.functional as TF

transform = transforms.Compose([transforms.Resize(65),
                                  transforms.CenterCrop(64),
                                  transforms.ToTensor()])
ds = {}
imglist = []
counter = 0
for subdir, dirs, files in os.walk(rootdir):
  if subdir.endswith("Im"):
    imagelist = imglist.copy()
    for file in files:
      period = file.index('.')
      dash = file.index('-')
      if file.endswith(".jpg"):
        image = Image.open(os.path.join(subdir, file))
        x = transform(image)
        x = x.unsqueeze(0)
        imagelist.append((x, file[dash+1:period]))

    if subdir[-16:-3].startswith("780688012670"):
      ds["9780688012670"] = imagelist
    else:
      ds[subdir[-16:-3]] = imagelist
    counter += 1
    print("Iteration: {}".format(counter))

In [ ]:
#import pickle
import torch
import re
#with open('drive/MyDrive/isbntoimg.pkl', 'rb') as fid:
#     ds = pickle.load(fid)
#with open('drive/MyDrive/idxtotext.pkl', 'rb') as fid:
#     idxtotext = pickle.load(fid)   
#helper.imshow(ds['9780688012670'][-1][0], normalize=False)
def embed_dict(idxtotext):
  for key in idxtotext.keys():
    embed = torch.zeros(0)
    
    text, isbn, idx = idxtotext[key]

    text = str(text)
    text = text.lower()
    text = text.strip('\n')
    text = text.strip('\r')
    text = re.split('[^a-zA-Z]', text)

    count = 0
    for wordidx in range(len(text)):
      if count == 10:
        break
      if text[wordidx] in embeddings_index.keys():
        embed = torch.cat([embed, torch.from_numpy(embeddings_index[text[wordidx]])], 0)
        count += 1
    if len(embed) != 1000:
      remaining = 1000 - len(embed)
      embed = torch.cat([embed, torch.zeros(remaining)], 0)
    assert len(embed) == 1000
    idxtotext[key] = embed, isbn, idx

embed_dict(idxtotext)


In [ ]:
with open('drive/MyDrive/embeddings_index.pkl', 'rb') as fid:
     embeddings_index = pickle.load(fid)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import re

#csvs["9780688012670"] = csvs["9780688012670 "]
#csvs.pop("9780688012670 ", None)
class ImageDatasetConditional(Dataset):
  def __init__(self, idxtotext, isbntoimg):
    self.idxtotext = idxtotext
    self.isbntoimg = isbntoimg
  def __len__(self):
    return len(idxtotext)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()    
    embed, isbn, item = self.idxtotext[idx]
    image = torch.ones((3, 64, 64))
    for elem in self.isbntoimg[isbn]:
      if elem[1] == str(item):
        image = elem[0]
    return {'image': image, 'text': embed}
dataset = ImageDatasetConditional(idxtotext, ds)
dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=64)

ValueError: ignored

In [ ]:

#!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/3bd7dea850e936d8cb44adda8200e4e2b5d627e3/intro-to-pytorch/helper.py

In [ ]:
#VANILLA GAN
import pickle
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np



class generator(nn.Module):
	def __init__(self):
		super(generator, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.noise_dim = 100
		self.ngf = 64

		# based on: https://github.com/pytorch/examples/blob/master/dcgan/main.py
		self.netG = nn.Sequential(
			nn.ConvTranspose2d(self.noise_dim, self.ngf * 8, 4, 1, 0, bias=False),
			nn.BatchNorm2d(self.ngf * 8),
			nn.ReLU(True),
			# state size. (ngf*8) x 4 x 4
			nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf * 4),
			nn.ReLU(True),
			# state size. (ngf*4) x 8 x 8
			nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf * 2),
			nn.ReLU(True),
			# state size. (ngf*2) x 16 x 16
			nn.ConvTranspose2d(self.ngf * 2,self.ngf, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf),
			nn.ReLU(True),
			# state size. (ngf) x 32 x 32
			nn.ConvTranspose2d(self.ngf, self.num_channels, 4, 2, 1, bias=False),
			nn.Tanh()
			 # state size. (num_channels) x 64 x 64
			)

	def forward(self, z):
		return self.netG(z)


class discriminator(nn.Module):
	def __init__(self):
		super(discriminator, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.ndf = 64

		self.B_dim = 128
		self.C_dim = 16

		self.netD_1 = nn.Sequential(
			# input is (nc) x 64 x 64
			nn.Conv2d(self.num_channels, self.ndf, 4, 2, 1, bias=False),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf) x 32 x 32
			nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 2),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*2) x 16 x 16
			nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 4),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*4) x 8 x 8
			nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 8),
			nn.LeakyReLU(0.2, inplace=True)
		)

		self.netD_2 = nn.Sequential(
			# state size. (ndf*8) x 4 x 4
			nn.Conv2d(self.ndf * 8, 1, 4, 1, 0, bias=False),
			nn.Sigmoid()
			)

	def forward(self, inp):
		x_intermediate = self.netD_1(inp)
		output =  self.netD_2(x_intermediate)
		return output.view(-1, 1).squeeze(1), x_intermediate


model = generator()
model.load_state_dict(torch.load('gengan_249.pth', map_location=torch.device('cpu')))
#x = torch.rand(1,100, 1, 1)
#out = model.forward(x)
#out = torch.from_numpy(out.detach().numpy())
#print(out)
#helper.imshow(out.squeeze(0), normalize=False)


#with open('ds2.pkl', 'rb') as fid:
#    ds2 = pickle.load(fid)

#min = 10000
#for key in ds2.keys():
  #sumval = torch.sum(abs(ds2[key] - out))
  #sumval = torch.linalg.norm(ds2[key] - out)
  #print(sumval)
  #if sumval < min:
    #copy = ds2[key]
    #min = sumval
  #print(ds2[key] - out)
#print(torch.std(copy- out))
#helper.imshow(copy, normalize=False)
#print(min)

imagez = torch.zeros((256, 3, 64, 64))
for i in range(256):
  x = torch.rand(1,100, 1, 1)
  out = model.forward(x)
  out = torch.from_numpy(out.detach().numpy())
  imagez[i] = out


In [ ]:
import torch
import torch.nn as nn
from torch import optim as optim
from torch.autograd import Variable
import numpy as np


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 50
#generator = generator()
#discriminator = discriminator()
discriminator.to(device)
generator.to(device)

num_epochs = 1

loss = nn.BCELoss()
optimD = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.9, 0.999))
optimG = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.9, 0.999))


l1_coef = 0
l2_coef = 0

criterion = nn.BCELoss()
l2_loss = nn.MSELoss()
l1_loss = nn.L1Loss()
iteration = 0

for epoch in range(num_epochs):
    for sample in dataloader:
        iteration += 1
        right_images = sample[0]

        right_images = Variable(right_images.float()).cuda()

        real_labels = torch.ones(right_images.size(0))
        fake_labels = torch.zeros(right_images.size(0))

        # ======== One sided label smoothing ==========
        # Helps preventing the discriminator from overpowering the
        # generator adding penalty when the discriminator is too confident
        # =============================================
        #smoothed_real_labels = torch.FloatTensor(Utils.smooth_label(real_labels.numpy(), -0.1))
        smoothed_real_labels = torch.FloatTensor(real_labels.numpy() - 0.1)
        real_labels = Variable(real_labels).cuda()
        smoothed_real_labels = Variable(smoothed_real_labels).cuda()
        fake_labels = Variable(fake_labels).cuda()


        # Train the discriminator
        discriminator.zero_grad()
        outputs, activation_real = discriminator(right_images)
        real_loss = criterion(outputs, smoothed_real_labels)
        real_score = outputs

        noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
        noise = noise.view(right_images.size(0), 100, 1, 1)
        fake_images = generator(noise)
        outputs, _ = discriminator(fake_images)
        fake_loss = criterion(outputs, fake_labels)
        fake_score = outputs

        d_loss = real_loss + fake_loss

        d_loss.backward()
        optimD.step()

        # Train the generator
        generator.zero_grad()
        noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
        noise = noise.view(right_images.size(0), 100, 1, 1)
        fake_images = generator(noise)
        outputs, activation_fake = discriminator(fake_images)
        _, activation_real = discriminator(right_images)

        activation_fake = torch.mean(activation_fake, 0)
        activation_real = torch.mean(activation_real, 0)

        # ======= Generator Loss function============
        # This is a customized loss function, the first term is the regular cross entropy loss
        # The second term is feature matching loss, this measure the distance between the real and generated
        # images statistics by comparing intermediate layers activations
        # The third term is L1 distance between the generated and real images, this is helpful for the conditional case
        # because it links the embedding feature vector directly to certain pixel values.
        g_loss = criterion(outputs, real_labels) \
                  + l2_coef * l2_loss(activation_fake, activation_real.detach()) \
                  + l1_coef * l1_loss(fake_images, right_images)

        g_loss.backward()
        optimG.step()

        if iteration % 5 == 0:
            #print("5 iterations done")
            print("D_loss = {}".format(d_loss))
            print("G loss = {}".format(g_loss))
            #self.logger.log_iteration_gan(epoch, d_loss, g_loss, real_score, fake_score)
            #self.logger.draw(right_images, fake_images)

    #self.logger.plot_epoch_w_scores(iteration)

    #if (epoch) % 50 == 0:
        #Utils.save_checkpoint(self.discriminator, self.generator, self.checkpoints_path, epoch)
    if epoch % 50 == 0:
      #SAVE
      #torch.save(model.state_dict(), PATH)

      #LOAD
      #model = TheModelClass(*args, **kwargs)
      #model.load_state_dict(torch.load(PATH))
      #model.eval()

      path = os.path.join(dir_path, subdir_path)
      torch.save(netD.state_dict(), '{0}/disc_{1}.pth'.format(path, epoch))
      torch.save(netG.state_dict(), '{0}/gen_{1}.pth'.format(path, epoch))

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-05-31 17:57:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-05-31 17:57:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-05-31 17:57:31--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
import numpy as np
print('Indexing word vectors.')

embeddings_index = {}
f = open('drive/MyDrive/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [ ]:
with open('embeddings_index.pkl', 'wb') as fid:
  pickle.dump(embeddings_index, fid)

In [ ]:

import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pdb
import pickle


class Concat_embed(nn.Module):

    def __init__(self, embed_dim, projected_embed_dim):
        super(Concat_embed, self).__init__()
        self.projection = nn.Sequential(
            nn.Linear(in_features=embed_dim, out_features=projected_embed_dim),
            nn.BatchNorm1d(num_features=projected_embed_dim),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
            )

    def forward(self, inp, embed):
        projected_embed = self.projection(embed)
        replicated_embed = projected_embed.repeat(4, 4, 1, 1).permute(2,  3, 0, 1)
        hidden_concat = torch.cat([inp, replicated_embed], 1)

        return hidden_concat



class condgenerator(nn.Module):
	def __init__(self):
		super(condgenerator, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.noise_dim = 100
		self.embed_dim = 1000
		self.projected_embed_dim = 128
		self.latent_dim = self.noise_dim + self.projected_embed_dim
		self.ngf = 64

		self.projection = nn.Sequential(
			nn.Linear(in_features=self.embed_dim, out_features=self.projected_embed_dim),
			nn.BatchNorm1d(num_features=self.projected_embed_dim),
			nn.LeakyReLU(negative_slope=0.2, inplace=True)
			)

		# based on: https://github.com/pytorch/examples/blob/master/dcgan/main.py
		self.netG = nn.Sequential(
			nn.ConvTranspose2d(self.latent_dim, self.ngf * 8, 4, 1, 0, bias=False),
			nn.BatchNorm2d(self.ngf * 8),
			nn.ReLU(True),
			# state size. (ngf*8) x 4 x 4
			nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf * 4),
			nn.ReLU(True),
			# state size. (ngf*4) x 8 x 8
			nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf * 2),
			nn.ReLU(True),
			# state size. (ngf*2) x 16 x 16
			nn.ConvTranspose2d(self.ngf * 2,self.ngf, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf),
			nn.ReLU(True),
			# state size. (ngf) x 32 x 32
			nn.ConvTranspose2d(self.ngf, self.num_channels, 4, 2, 1, bias=False),
			nn.Tanh()
			 # state size. (num_channels) x 64 x 64
			)


	def forward(self, embed_vector, z):

		projected_embed = self.projection(embed_vector).unsqueeze(2).unsqueeze(3)
		latent_vector = torch.cat([projected_embed, z], 1)
		output = self.netG(latent_vector)

		return output

class conddiscriminator(nn.Module):
	def __init__(self):
		super(conddiscriminator, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.embed_dim = 1000
		self.projected_embed_dim = 128
		self.ndf = 64
		self.B_dim = 128
		self.C_dim = 16

		self.netD_1 = nn.Sequential(
			# input is (nc) x 64 x 64
			nn.Conv2d(self.num_channels, self.ndf, 4, 2, 1, bias=False),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf) x 32 x 32
			nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 2),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*2) x 16 x 16
			nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 4),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*4) x 8 x 8
			nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 8),
			nn.LeakyReLU(0.2, inplace=True),
		)

		self.projector = Concat_embed(self.embed_dim, self.projected_embed_dim)

		self.netD_2 = nn.Sequential(
			# state size. (ndf*8) x 4 x 4
			nn.Conv2d(self.ndf * 8 + self.projected_embed_dim, 1, 4, 1, 0, bias=False),
			nn.Sigmoid()
			)	

	def forward(self, inp, embed):
		x_intermediate = self.netD_1(inp)
		x = self.projector(x_intermediate, embed)
		x = self.netD_2(x)

		return x.view(-1, 1).squeeze(1) , x_intermediate



model = condgenerator()
model.load_state_dict(torch.load('gen_190.pth', map_location=torch.device('cpu')))
x = torch.rand(10, 100, 1, 1)
#y = torch.zeros(10,1000)
with open("drive/MyDrive/embeddings_index.pkl", "rb") as fp:
  embeddings_index= pickle.load(fp)
vec = embeddings_index["the"]
vec2 = embeddings_index["clear"]
vec4 = embeddings_index["blue"]
vec5 = embeddings_index["sky"]

y = torch.cat([torch.tensor(vec), torch.tensor(vec2), torch.tensor(vec3), torch.tensor(vec4),torch.zeros(600)], 0)
y = y.unsqueeze(0)
print(y.shape)
tens = torch.zeros((10, 1000))
tens += y
#tens = torch.rand(10, 1000)
#tens[0] = y
out = model(tens, x)
out = torch.from_numpy(out.detach().numpy())
#print(out)
print(out.shape)
helper.imshow(out[0], normalize=False)

NameError: ignored

In [ ]:
import torch
import torch.nn as nn
from torch import optim as optim
from torch.autograd import Variable
import numpy as np


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 50
generator = condgenerator()
discriminator = conddiscriminator()
discriminator.to(device)
generator.to(device)

num_epochs = 1

loss = nn.BCELoss()
optimD = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.9, 0.999))
optimG = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.9, 0.999))


l1_coef = 50
l2_coef = 100



criterion = nn.BCELoss()
l2_loss = nn.MSELoss()
l1_loss = nn.L1Loss()
iteration = 0
g_losses = []
d_losses = []
for epoch in range(num_epochs):
    for sample in dataloader:
        iteration += 1
        right_images = sample['image']
        right_embed = sample['text']


        right_images = Variable(right_images.float())
        right_embed = Variable(right_embed.float())

        real_labels = torch.ones(right_images.size(0))
        fake_labels = torch.zeros(right_images.size(0))

        # ======== One sided label smoothing ==========
        # Helps preventing the discriminator from overpowering the
        # generator adding penalty when the discriminator is too confident
        # =============================================
        #smoothed_real_labels = torch.FloatTensor(Utils.smooth_label(real_labels.numpy(), -0.1))
        smoothed_real_labels = torch.FloatTensor(real_labels.numpy() - 0.1)

        real_labels = Variable(real_labels)
        smoothed_real_labels = Variable(smoothed_real_labels)
        fake_labels = Variable(fake_labels)

        # Train the discriminator
        discriminator.zero_grad()
        outputs, activation_real = discriminator(right_images, right_embed)
        real_loss = criterion(outputs, smoothed_real_labels)
        real_score = outputs


        noise = Variable(torch.randn(right_images.size(0), 100))
        noise = noise.view(noise.size(0), 100, 1, 1)
        fake_images = generator(right_embed, noise)
        outputs, _ = discriminator(fake_images, right_embed)
        fake_loss = criterion(outputs, fake_labels)
        fake_score = outputs

        d_loss = real_loss + fake_loss


        d_loss.backward()
        optimD.step()

        # Train the generator
        generator.zero_grad()
        noise = Variable(torch.randn(right_images.size(0), 100))
        noise = noise.view(noise.size(0), 100, 1, 1)
        fake_images = generator(right_embed, noise)
        outputs, activation_fake = discriminator(fake_images, right_embed)
        _, activation_real = discriminator(right_images, right_embed)

        activation_fake = torch.mean(activation_fake, 0)
        activation_real = torch.mean(activation_real, 0)


        #======= Generator Loss function============
        # This is a customized loss function, the first term is the regular cross entropy loss
        # The second term is feature matching loss, this measure the distance between the real and generated
        # images statistics by comparing intermediate layers activations
        # The third term is L1 distance between the generated and real images, this is helpful for the conditional case
        # because it links the embedding feature vector directly to certain pixel values.
        #===========================================
        g_loss = criterion(outputs, real_labels) \
                  + l2_coef * l2_loss(activation_fake, activation_real.detach()) \
                  + l1_coef * l1_loss(fake_images, right_images)

        g_loss.backward()
        optimG.step()

        if iteration % 5 == 0:
          g_losses.append(g_loss)
          d_losses.append(d_loss)
        #    self.logger.log_iteration_gan(epoch,d_loss, g_loss, real_score, fake_score)
        #    self.logger.draw(right_images, fake_images)

    #self.logger.plot_epoch_w_scores(epoch)

        if (iteration) % 5 == 0:
          with open("d_losses{}.txt".format(epoch), "wb") as fp:
            pickle.dump(d_losses, fp)
          with open("g_losses{}.txt".format(epoch), "wb") as fp:
            pickle.dump(g_losses, fp)
        #Utils.save_checkpoint(self.discriminator, self.generator, self.checkpoints_path, self.save_path, epoch)

KeyboardInterrupt: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from tqdm import trange
import numpy as np


class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,      # First max pooling features
        192: 1,     # Second max pooling featurs
        768: 2,     # Pre-aux classifier features
        2048: 3,    # Final average pooling features
        'prob': 4,  # softmax layer
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False,
                 use_fid_inception=True):
        """Build pretrained InceptionV3
        Parameters
        ----------
        output_blocks : list of int
            Indices of blocks to return features of. Possible values are:
                - 0: corresponds to output of first max pooling
                - 1: corresponds to output of second max pooling
                - 2: corresponds to output which is fed to aux classifier
                - 3: corresponds to output of final average pooling
        resize_input : bool
            If true, bilinearly resizes input to width and height 299 before
            feeding input to model. As the network without fully connected
            layers is fully convolutional, it should be able to handle inputs
            of arbitrary size, so resizing might not be strictly needed
        normalize_input : bool
            If true, scales the input from range (0, 1) to the range the
            pretrained Inception network expects, namely (-1, 1)
        requires_grad : bool
            If true, parameters of the model require gradients. Possibly useful
            for finetuning the network
        use_fid_inception : bool
            If true, uses the pretrained Inception model used in Tensorflow's
            FID implementation. If false, uses the pretrained Inception model
            available in torchvision. The FID Inception model has different
            weights and a slightly different structure from torchvision's
            Inception model. If you want to compute FID scores, you are
            strongly advised to set this parameter to true to get comparable
            results.
        """
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        # assert self.last_needed_block <= 3, \
        #     'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        #if use_fid_inception:
        #    inception = fid_inception_v3()
        #else:
        #    inception = models.inception_v3(
         #       pretrained=True, init_weights=False)

        inception = models.inception_v3(pretrained=True, init_weights=False)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        if self.last_needed_block >= 4:
            self.fc = inception.fc
            self.fc.bias = None

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        if self.last_needed_block >= 4:
            x = F.dropout(x, training=self.training)
            # N x 2048 x 1 x 1
            x = torch.flatten(x, 1)
            # N x 2048
            x = self.fc(x)
            x = F.softmax(x, dim=1)
            outp.append(x)

        return outp



def get_inception_score(images, splits=10, batch_size=32, use_torch=False,
                        verbose=False, parallel=False):
    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM['prob']
    model = InceptionV3([block_idx]).to(device)
    model.eval()

    if parallel:
        model = torch.nn.DataParallel(model)

    preds = []
    iterator = trange(
        0, len(images), batch_size, dynamic_ncols=True, leave=False,
        disable=not verbose, desc="get_inception_score")

    for start in iterator:
        end = start + batch_size
        batch_images = images[start: end]
        batch_images = torch.from_numpy(batch_images).type(torch.FloatTensor)
        batch_images = batch_images.to(device)
        with torch.no_grad():
            pred = model(batch_images)[0]
        if use_torch:
            preds.append(pred)
        else:
            preds.append(pred.cpu().numpy())
    if use_torch:
        preds = torch.cat(preds, 0)
    else:
        preds = np.concatenate(preds, 0)
    scores = []
    for i in range(splits):
        part = preds[
            (i * preds.shape[0] // splits):
            ((i + 1) * preds.shape[0] // splits), :]
        if use_torch:
            kl = part * (
                torch.log(part) -
                torch.log(torch.unsqueeze(torch.mean(part, 0), 0)))
            kl = torch.mean(torch.sum(kl, 1))
            scores.append(torch.exp(kl))
        else:
            kl = part * (
                np.log(part) -
                np.log(np.expand_dims(np.mean(part, 0), 0)))
            kl = np.mean(np.sum(kl, 1))
            scores.append(np.exp(kl))
    if use_torch:
        scores = torch.stack(scores)
        is_mean, is_std = (
            torch.mean(scores).cpu().item(), torch.std(scores).cpu().item())
    else:
        is_mean, is_std = np.mean(scores), np.std(scores)
    del preds, scores, model
    return is_mean, is_std

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#var = torch.rand(10, 3, 64, 64)
#var[:7] = imagez[:] 
#print(imagez[0].shape)

#mean, std = get_inception_score(np.array())
#print(imagez.shape)

mean, std = get_inception_score((np.array(imagez)))
print(mean)
print(std)

1.4694201
0.06556692


In [ ]:
import torch
counter = 0
ds2 = {}
counter = 0
for key in ds:
  for elem in ds[key]:
    if torch.count_nonzero(elem[0]- 1) > 3072:
      ds2[counter] = elem[0]
      counter += 1
 # ds2[counter] = include

In [ ]:
with open('ds2.pkl', 'wb') as fid:
     pickle.dump(ds2, fid)

tensor([[[0.9294, 0.6863, 0.4275,  ..., 1.0000, 1.0000, 1.0000],
         [0.9333, 0.6118, 0.2118,  ..., 1.0000, 1.0000, 1.0000],
         [0.9333, 0.7294, 0.5569,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.8824, 0.8863, 0.8627,  ..., 0.8039, 0.9529, 0.9961],
         [0.8824, 0.8824, 0.8627,  ..., 0.8000, 0.9686, 0.9961],
         [0.8745, 0.8745, 0.8627,  ..., 0.8510, 0.9647, 0.9961]],

        [[0.9529, 0.7882, 0.5451,  ..., 1.0000, 1.0000, 1.0000],
         [0.9569, 0.7529, 0.4667,  ..., 1.0000, 1.0000, 1.0000],
         [0.9647, 0.8000, 0.5373,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.8863, 0.8863, 0.8667,  ..., 0.8000, 0.9569, 1.0000],
         [0.8824, 0.8824, 0.8275,  ..., 0.7961, 0.9725, 1.0000],
         [0.8784, 0.8784, 0.8627,  ..., 0.7647, 0.9529, 1.0000]],

        [[0.8941, 0.6745, 0.4118,  ..., 1.0000, 0.9922, 0.9922],
         [0.8902, 0.6392, 0.1686,  ..., 1.0000, 1.0000, 0.9961],
         [0.8980, 0.7373, 0.2824,  ..., 1.0000, 1.0000, 1.

In [ ]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
import tensorflow as tf 
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
 
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
# define two fake collections of images
images1 = randint(0, 255, 256*32*32*3)
images1 = images1.reshape((256,32,32,3))
images2 = randint(0, 255, 256*32*32*3)
images2 = images2.reshape((256,32,32,3))
print('Prepared', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# fid between images1 and images1
fid = calculate_fid(model, images1, images1)
print('FID (same): %.3f' % fid)
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)

Prepared (256, 32, 32, 3) (256, 32, 32, 3)
Scaled (256, 299, 299, 3) (256, 299, 299, 3)
FID (same): -0.000
FID (different): 7.363


In [ ]:

#with open('ds2.pkl', 'rb') as fid:
#    ds2 = pickle.load(fid)
#images1 = ds2[:2]
images1 = randint((2, 3, 64, 64))
images1[0] = ds2[0]
images1[1] = ds2[1]
#images1 = randint((2,32,32,3))

#images1 = images1.reshape((1, 32, 32, 3))
#images1 = images1.unsqueeze(0)
images1 = images1.permute(0, 3, 1, 2 )
images2 = imagez[:2]
#images2 = images2.reshape((1, 32, 32, 3))
#images2 = images2.unsqueeze(0)
images2 = images2.permute(0, 3, 1, 2 )

images1 = scale_images(images1, (299, 299, 3))
images2 = scale_images(images2, (299, 299, 3))
images1 = tf.convert_to_tensor(preprocess_input(images1))
images2 = tf.convert_to_tensor(preprocess_input(images2))
print(images1.shape)
print(images2.shape)
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)



ValueError: ignored

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from torch import optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle

class generator2(nn.Module):
	def __init__(self):
		super(generator2, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.noise_dim = 100
		self.ngf = 64

		# based on: https://github.com/pytorch/examples/blob/master/dcgan/main.py
		self.netG = nn.Sequential(


			nn.ConvTranspose2d(self.noise_dim, self.ngf * 8, 4, 1, 0, bias=False),
			nn.BatchNorm2d(self.ngf * 8),
			nn.ReLU(True),
			# state size. (ngf*8) x 4 x 4
			nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf * 4),
			nn.ReLU(True),
			# state size. (ngf*4) x 8 x 8
			nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf * 2),
			nn.ReLU(True),
			# state size. (ngf*2) x 16 x 16
			nn.ConvTranspose2d(self.ngf * 2,self.ngf, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ngf),
			nn.ReLU(True),
			# state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(self.ngf, self.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),
            # (ngf x 64 x 64)
            nn.ConvTranspose2d(self.ngf , self.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),
            # (ngf x 128 x 128)
			nn.ConvTranspose2d(self.ngf, self.num_channels, 4, 2, 1, bias=False),
			nn.Tanh()
			 # state size. (num_channels) x 256 x 256
			)

	def forward(self, z):
		return self.netG(z)


class discriminator2(nn.Module):
	def __init__(self):
		super(discriminator2, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.ndf = 64

		self.B_dim = 128
		self.C_dim = 16

		self.netD_1 = nn.Sequential(
            #input is (nc) x 256 x 256
            nn.Conv2d(self.num_channels, self.ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            #input is (ndf) x 128 x 128
            nn.Conv2d(self.ndf, self.ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
			# input is (ndf) x 64 x 64
			nn.Conv2d(self.ndf, self.ndf, 4, 2, 1, bias=False),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf) x 32 x 32
			nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 2),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*2) x 16 x 16
			nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 4),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*4) x 8 x 8
			nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 8),
			nn.LeakyReLU(0.2, inplace=True)
		)

		self.netD_2 = nn.Sequential(
			# state size. (ndf*8) x 4 x 4
			nn.Conv2d(self.ndf * 8, 1, 4, 1, 0, bias=False),
			nn.Sigmoid()
			)

	def forward(self, inp):
		x_intermediate = self.netD_1(inp)
		output =  self.netD_2(x_intermediate)
		return output.view(-1, 1).squeeze(1), x_intermediate



model = generator2()
model.load_state_dict(torch.load('gengannew_49.pth', map_location=torch.device('cpu')))
x = torch.rand(1,100, 1, 1)
out = model.forward(x)
out = torch.from_numpy(out.detach().numpy())
#print(out)
helper.imshow(out.squeeze(0), normalize=False)